In [1]:
# Import the needed libraries
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
# Set the random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
# Load the dataset to be used for classification (naming it 'training') 
#!unzip dataset.zip
#input_folder = '/kaggle/input/training/training'

In [4]:
# Split the training dataset between train (85%) and validation set (15%)
#import splitfolders
#splitfolders.ratio("input_folder", output="output", seed=1234, ratio=(.85, .15))

In [5]:
# Load the folders obtained above (naming it 'splitfolder') and define the datasets folders:
dataset_dir_train = '/kaggle/input/splitfolder/output/train'
dataset_dir_val = '/kaggle/input/splitfolder/output/val'

In [9]:
# Plot example images from dataset
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
  if i < len(labels):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir_train, labels[i])))[2]
    class_img = class_imgs[0]
    img = Image.open('{}/{}/{}'.format(dataset_dir_train, labels[i], class_img))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation: since we just want to apply it to training data, we create two different ImageDataGenerator, one for the 
# training set and the other for the validation set 
datagen_train = ImageDataGenerator(
                                   horizontal_flip=True,
                                   vertical_flip = True,
                                   zoom_range = 0.4,
                                   rotation_range = 45,
                                   height_shift_range=50,
                                   fill_mode='reflect',
                                   preprocessing_function= tf.keras.applications.efficientnet.preprocess_input
                                  )   
datagen_val = ImageDataGenerator( preprocessing_function= tf.keras.applications.efficientnet.preprocess_input)


train_generator_aug = datagen_train.flow_from_directory(
    directory= dataset_dir_train,
    target_size=(256,256),
    color_mode='rgb',
    classes= None,
    class_mode= 'categorical',
    shuffle= True,
    seed=seed
)

val_generator_aug = datagen_val.flow_from_directory(
    directory=dataset_dir_val,
    target_size=(256,256),
    color_mode='rgb',
    classes= None,
    class_mode= 'categorical',
    shuffle= True,
    seed = seed
)

In [13]:
# Inspection of the labels 

fig, ax = plt.subplots()
# We need to draw the canvas, otherwise the labels won't be positioned and won't have values yet.
fig.canvas.draw()
ax.hist(train_generator_aug.labels, bins=14)
ax.set_xticks(range(14))
ax.set_xticklabels(labels, rotation='vertical')
plt.show()

In [14]:
# The histogram above shows unbalance among classes; therefore, we apply weights in order to balance the dataset. 
from sklearn.utils import class_weight 
import numpy as np

class_weights = class_weight.compute_class_weight(
           'balanced',
            classes=np.unique(train_generator_aug.classes), 
            y=train_generator_aug.classes)

train_class_weights = dict(enumerate(class_weights))


## TRANSFER LEARNING
We decided to use an EfficientNetB6 as pre-trained architecture

In [ ]:
#!pip install --upgrade requests

In [15]:
# Load the chosen supernet: 
supernet = tfk.applications.efficientnet.EfficientNetB6(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
    )

In [ ]:
# A summary of the supernet and the corresponding plot
supernet.summary()
tfk.utils.plot_model(supernet)

In [20]:
# Define batch_size and number of epochs: 
batch_size = 128
epochs = 100

In [21]:
# Use the supernet as feature extractor
supernet.trainable = False  

inputs = tfk.Input(shape=(256,256,3))
x = supernet(inputs)
x = tfkl.GlobalAveragePooling2D(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)

outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), 
                  optimizer=tfk.optimizers.Adam(),
                  metrics='accuracy')
tl_model.summary() #there will be non trainable parameters

In [ ]:
# Train the model
history = tl_model.fit(
    x = train_generator_aug,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = val_generator_aug,
    callbacks =  [tfk.callbacks.EarlyStopping(monitor='val_accuracy',  
                                             mode='max',   
                                             patience=10,
                                             restore_best_weights=True) 
                 ],
    class_weight = train_class_weights
).history


In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
#plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
#plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save best epoch model
tl_model.save('EfficientNetB6Model')

## FINE TUNING
We applied fine tuning in order to have the pre-trained network recognize classes it was not originally trained on.

In [ ]:
# The model saved in the above line should be added to the data as input (naming it'efficientnet') and then re-loaded: 
ft_model = tfk.models.load_model('/kaggle/input/efficientnet/EfficientNetB6Model')

In [ ]:
# Set all  layers to True
ft_model.get_layer('efficientnetb6').trainable = True
for i, layer in enumerate(ft_model.get_layer('efficientnetb6').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first 400 layers 
for i, layer in enumerate(ft_model.get_layer('efficientnetb6').layers[:400]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetb6').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = train_generator_aug,
    batch_size = 128,
    epochs = 50,
    validation_data = val_generator_aug,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)],
    class_weight=train_class_weights
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
#plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
#plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
#plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save the model
ft_model.save('EfficientNetB6_FineTuning')